In [1]:
import sys
sys.path.append("/home/kieran/code/Ant-mel/legendary_game_recs/")

from preprocessing.preprocess_1_cleaning import *
from preprocessing.preprocess_2_features import *
from preprocessing.pipeline_the_unification import *

In [2]:
raw_data = pd.read_csv("../raw_data/reference_csv_v1.csv", low_memory=False)

In [4]:
raw_data

,title,release_date,plays,playing,backlogs,wishlist,developers,avg_review,genres,platforms,...,Shooter,Strategy,Platform,Arcade,Sport,gen_1,gen_2,gen_3,link,game_id
0,Darling Duality: Winter Wish,2024-12-31,7,0,2,2,['Melancholy Marionette'],2.2,['Visual Novel'],"['Windows PC', 'Mac', 'Linux']",...,0,0,0,0,0,0,0,1,/games/darling-duality-winter-wish/,178780.0
1,Anger Foot,2024-12-31,80,1,82,297,"['Devolver Digital', 'Robbie Fraser']",3.3,"['Brawler', 'Indie', 'Shooter']",['Windows PC'],...,1,0,0,0,0,0,0,1,/games/anger-foot/,147184.0
2,Life Is Feudal: MMO,2023-12-31,9,0,5,1,['Long Tale Games'],2.4,"['Adventure', 'RPG', 'Simulator']",['Windows PC'],...,0,0,0,0,0,0,0,1,/games/life-is-feudal-mmo/,24611.0
3,The Making of Karateka,2023-08-29,10,3,9,41,['Digital Eclipse'],4.5,"['Adventure', 'Fighting', 'Platform']","['Windows PC', 'PlayStation 4', 'Xbox One', 'P...",...,0,0,1,0,0,0,0,1,/games/the-making-of-karateka/,256293.0
4,Enigma of Fear,2023-12-31,16,0,26,155,['Dumativa Game Studio'],2.1,"['Adventure', 'Indie', 'Puzzle']",['Windows PC'],...,0,0,0,0,0,0,0,1,/games/enigma-of-fear/,140133.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36845,Final Crisis: Terrestrial Defense Police,1991-12-01,4,0,1,4,['Techno Grard'],3.8,['Shooter'],['PC-8801'],...,1,0,0,0,0,1,0,0,/games/final-crisis/,141644.0
36846,Level Up!,2009-11-25,6,0,0,1,[''],2.7,"['Adventure', 'Indie']",['Web browser'],...,0,0,0,0,0,0,1,0,/games/level-up--1/,95373.0
36847,Sea Battle,2014-02-07,3,0,0,0,['BYRIL'],2.2,[''],"['Android', 'iOS']",...,0,0,0,0,0,0,0,1,/games/sea-battle--2/,134381.0
36848,We Are Illuminati,2017-11-02,18,0,0,0,['Tapps Games'],2.0,"['Indie', 'Simulator', 'Strategy']","['Android', 'iOS']",...,0,1,0,0,0,0,0,1,/games/we-are-illuminati/,120333.0


In [5]:
raw_data['description'] = raw_data['description'].astype(str)

In [6]:
extended_stop_words = ['able','access','across','also','always','another','away','back','become','best','better','big','box','bring','certain','clear','close','come','console','content','could','course','digital','dont','one','two','three','four','five','six','seven','eight','nine','ten','hundred','thousand','either','enjoy','enough','even','exclusive','extra','feature','franchise','full','fully','fun','game','gameplay','genre','get','give','go','good','great','great','greatest','happen','however','huge','ii','improve','include','increase','inside','interactive','introduce','instead','involve','know','large','last','later','launch','lead','let','level','like','little','look','long','main','may','meet','might','mix','modern','mode','much','must','nan','name','new','next','need','number','nintendo','official','offer','object','option','order','original','originally','others','part','pc','perfect','platform','play','playable','player','playstation','plus','possible','port','prepare','previous','progress','project','publish','put','reach','ready','remain','return','screen','scroll','second','first','third','see','sega','sequel','series','set','several','show','side','similar','since','small','something','sound','special','start','stat','state','stay','still','studio','super','take','switch','tell','test','th','though','throughout','title','together','top','try','ultimate','unique','update','upon','us','use','version','via','wait','want','wii','within','without','would','xbox','youll','youre','youve']

In [7]:
import string
from nltk.corpus import stopwords 
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer

stop_words = stopwords.words('english') ## define stopwords
print(len(stop_words))
stop_words.extend(extended_stop_words)
print(len(stop_words))

179
360


In [8]:
import string
from nltk.corpus import stopwords 
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer

def preprocessing(sentence):
    
    # Basic cleaning
    sentence = sentence.strip() ## remove whitespaces
    sentence = sentence.lower() ## lowercase 
    sentence = ''.join(char for char in sentence if not char.isdigit()) ## remove numbers
    
    # Advanced cleaning
    for punctuation in string.punctuation:
        sentence = sentence.replace(punctuation, '') ## remove punctuation
    
    tokenized_sentence = word_tokenize(sentence) ## tokenize 

    tokenized_sentence_cleaned = [ ## remove stopwords
        w for w in tokenized_sentence if not w in stop_words
    ]

    
    noun_lemmatized = [
        WordNetLemmatizer().lemmatize(word, pos = "n") 
        for word in tokenized_sentence_cleaned
    ]
    lemmatized = [
        WordNetLemmatizer().lemmatize(word, pos = "v") 
        for word in noun_lemmatized
    ]
    
    lemmatized_sentence_cleaned = [ ## remove stopwords
        w for w in lemmatized if not w in stop_words
    ]
    
    cleaned_sentence = ' '.join(word for word in lemmatized_sentence_cleaned)
    
    return cleaned_sentence

In [9]:
raw_data['clean_text'] = raw_data['description'].apply(preprocessing)

In [10]:
raw_data['clean_text'][4033]

'wild case firstperson point click adventure storybased adventure character encounter various event require solve puzzle'

In [11]:
preprocessing(raw_data['clean_text'][4033])

'wild case firstperson point click adventure storybased adventure character encounter various event require solve puzzle'

In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(min_df=0.02)

vectorized_descriptions = vectorizer.fit_transform(raw_data['clean_text'])
vectorized_descriptions = pd.DataFrame(
    vectorized_descriptions.toarray(), 
    columns = vectorizer.get_feature_names_out()
)

vectorized_descriptions

,ability,action,add,advance,adventure,alien,allow,along,ancient,arcade,...,visual,war,way,weapon,well,win,work,world,year,young
0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.00000,0.0
1,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.00000,0.0
2,0.284665,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.279566,0.0,0.0,0.181832,0.00000,0.0
3,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.00000,0.0
4,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.00000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36845,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.57374,0.0
36846,0.273299,0.0,0.000000,0.0,0.0,0.0,0.0,0.290768,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.00000,0.0
36847,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.153058,0.00000,0.0
36848,0.000000,0.0,0.612331,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.433409,0.0,0.000000,0.0,0.0,0.000000,0.00000,0.0


In [19]:
from sklearn.decomposition import LatentDirichletAllocation

# Instantiate the LDA 

lda_model = LatentDirichletAllocation(n_components=10, max_iter = 40, learning_method='online',)

# Fit the LDA on the vectorized documents
lda_model.fit(vectorized_descriptions)

LatentDirichletAllocation(learning_method='online', max_iter=40)

In [20]:
def print_topics(model, vectorizer):
    for idx, topic in enumerate(model.components_):
        print("Topic %d:" % (idx))
        print([(vectorizer.get_feature_names_out()[i], topic[i])
                        for i in topic.argsort()[:-10 - 1:-1]])

In [21]:
print_topics(lda_model, vectorizer)

Topic 0:
[('war', 578.7124094912408), ('japan', 549.4600178891938), ('star', 374.4409034243144), ('point', 350.4988049761372), ('music', 340.62333845764425), ('begin', 322.916220300841), ('hit', 315.0709150763027), ('goal', 311.01222156720365), ('year', 302.37943388581454), ('time', 295.668807076177)]
Topic 1:
[('puzzle', 780.0061293815988), ('collect', 434.0230797353847), ('defeat', 391.1820579712103), ('help', 385.5560540123098), ('world', 374.3263949984754), ('quest', 346.28480603727417), ('adventure', 343.31981476486766), ('dark', 334.20139527667266), ('item', 333.4940924057921), ('solve', 319.7819600546812)]
Topic 2:
[('enemy', 690.5361094301263), ('battle', 644.0909583081843), ('fight', 634.3113373306027), ('weapon', 518.1398791377164), ('attack', 462.326767133689), ('combat', 447.5251447784858), ('control', 407.6849239673328), ('mission', 381.0443268840048), ('stage', 370.9988478699374), ('power', 338.9666883730401)]
Topic 3:
[('race', 654.9583479548076), ('girl', 417.7999792130

In [22]:
raw_data[['clean_text']]

,clean_text
0,multiple pick poison choose sweet sinister yan...
1,anger foot lightningfast hardbass blast kick d...
2,life feudal mmo openworld survival epic journe...
3,history behind scene jordan mechners landmark ...
4,mia paranormal detective search miss father pe...
...,...
36845,year ad galactic federal government agree full...
36846,create titchuk release flash portal november a...
36847,sea battle board everybody love childhood onli...
36848,control mind insert subliminal message video c...


In [ ]:
vectorized_example = raw_data['clean_text'][36819]

In [ ]:
vectorized_example

In [ ]:
ve2 = vectorizer.transform(pd.Series(vectorized_example))
ve2

In [ ]:
vec_array = lda_model.transform(ve2)

In [ ]:
vec_array[0].tolist()

In [ ]:
vec_array[0].tolist().index(max(vec_array[0].tolist()))

In [23]:
def nlp_topic(x):
    vectorized_x = vectorizer.transform(pd.Series(x))
    vec_x_array = lda_model.transform(vectorized_x)
    vec_list = vec_x_array[0].tolist()
    return vec_list.index(max(vec_list))
    

In [24]:
raw_data['topic'] = raw_data['clean_text'].apply(nlp_topic)

/home/kieran/.pyenv/versions/3.10.6/envs/lewagon/lib/python3.10/site-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but LatentDirichletAllocation was fitted with feature names
  warnings.warn(


In [25]:
raw_data[['topic']].value_counts()

topic
1        5630
0        5604
2        5463
6        4884
5        4795
9        2929
3        2370
8        2142
4        1663
7        1370
dtype: int64

In [26]:
raw_data[['topic']]

,topic
0,9
1,0
2,5
3,7
4,1
...,...
36845,2
36846,1
36847,9
36848,0


In [27]:
pd.get_dummies(raw_data['topic'])

,0,1,2,3,4,5,6,7,8,9
0,0,0,0,0,0,0,0,0,0,1
1,1,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,1,0,0,0,0
3,0,0,0,0,0,0,0,1,0,0
4,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
36845,0,0,1,0,0,0,0,0,0,0
36846,0,1,0,0,0,0,0,0,0,0
36847,0,0,0,0,0,0,0,0,0,1
36848,1,0,0,0,0,0,0,0,0,0


In [30]:
topics_for_36k_games = pd.get_dummies(raw_data['topic'])

In [31]:
topics_for_36k_games.to_csv("../raw_data/final_topics.csv", index=False)